This notebook computes an a posteriori distribution for the steel sector in the US. Please, start with the second part where the normalizing constant is computed.

In [1]:

import pandas as pd
import numpy as np
import src.utils

In [3]:
from GSPTDataset import GSPTDataset
gspt = GSPTDataset(data_path="Global-Steel-Plant-Tracker-2023-03-2.xlsx",
                        missing_years_path="filled_missing_start_years.xlsx",
                        version_year=2023,
                       gspt2gspt_path="GSPT2GSPT.json")

from functions import get_cartesian, coordinates_change, f, transform_coordinates_to_lat_long


In [ ]:
df_gspt= gspt.get_operating_plants(start_year=2019,melt=True)
df_gspt=df_gspt.drop_duplicates(["Coordinates"])


In [5]:
steel_coordinates=coordinates_change(df_gspt)
tuple_steel=[tuple(c) for c in steel_coordinates]

In [6]:
import tqdm
from joblib import Parallel,delayed
from tqdm import tqdm
from functions import geo
from geopy.distance import geodesic,great_circle
from functions import geo_great_circle,geo, list_coordinates, geo_great_circle, geo_great_circle_gaussian_kernel

In [7]:
"""We start with the USA LitPop database with 5 minutes resolution"""
usa_litpop=pd.read_hdf("C:\\Users\\Denisa.draghia\\Downloads\\climada_python-1.2.0\\climada_python-1.2.0\\data\\results\\5 minutes\\LitPop_pc_300_arcsec_USA_v1.hdf5")
usa_litpop=usa_litpop.drop(columns=['geometry','region_id','impf_'])
usa_litpop_coordinates=usa_litpop[['latitude','longitude']]

In [8]:
list_usa=[tuple(row) for row in usa_litpop_coordinates.to_records(index=False)]

In [9]:
funk= lambda x: geo_great_circle(lat_lon=x,h=100, tuple_steel=tuple_steel)
a=Parallel(n_jobs=-1)(delayed(funk)(c) for c in tqdm(list_usa))

100%|██████████| 160785/160785 [05:15<00:00, 510.26it/s]


In [12]:
usa_litpop['bayesian_update']=pd.DataFrame(a)
usa_litpop['numerator']=np.where(usa_litpop['value'] == 0, 0, np.exp(np.log(usa_litpop['value']) + usa_litpop['bayesian_update'] ))
usa_litpop['a_posteriori_distribution']=usa_litpop['numerator']/98873995047.30652

c:\Users\Denisa.draghia\.virtualenvs\Denisa.draghia-IwxA2Vx_\lib\site-packages\pandas\core\arraylike.py:396: RuntimeWarning: divide by zero encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)


In [14]:
usa_litpop

,value,latitude,longitude,bayesian_update,numerator,a_posteriori_distribution
0,0.000000,51.375000,-178.958333,-9.659901,0.000000,0.000000e+00
1,0.000000,51.875000,-178.208333,-9.668922,0.000000,0.000000e+00
2,0.000000,51.875000,-178.125000,-9.672842,0.000000,0.000000e+00
3,0.000000,51.875000,-178.041667,-9.676758,0.000000,0.000000e+00
4,0.000000,51.875000,-177.958333,-9.680673,0.000000,0.000000e+00
...,...,...,...,...,...,...
160780,0.000000,52.875000,173.041667,-9.199766,0.000000,0.000000e+00
160781,4348.029150,52.875000,173.125000,-9.203852,0.437633,4.426171e-12
160782,29688.926871,52.875000,173.208333,-9.207936,2.976040,3.009932e-11
160783,0.000000,52.791667,172.791667,-9.191016,0.000000,0.000000e+00


In [ ]:
'''It plots the a posteriori distribution obtained using an exponential kernel with parameter h=100'''
import plotly.express as px
fig = px.density_mapbox(usa_litpop, lat='latitude', lon='longitude', z='a_posteriori_distribution', radius=10,
                        center=dict(lat=0, lon=180), zoom=0,
                        mapbox_style="stamen-terrain")
fig.show()

In [ ]:
import plotly.express as px
fig = px.density_mapbox(usa_litpop, lat='latitude', lon='longitude', z='value', radius=10,
                        center=dict(lat=0, lon=180), zoom=0,
                        mapbox_style="stamen-terrain")
fig.show()

Compute normalizing constant

In [16]:
world=pd.read_csv("C:\\Users\\Denisa.draghia\\Downloads\\climada_python-1.2.0\\climada_python-1.2.0\\data\\results\\world_5 minutes.csv")

In [11]:
list_world=list_coordinates(world)

In [ ]:
funk= lambda x: geo_great_circle(lat_lon=x,h=100, tuple_steel=tuple_steel)
a=Parallel(n_jobs=-1)(delayed(funk)(c) for c in tqdm(list_world))

In [43]:
world['bayesian_update']=pd.DataFrame(a)
world['numerator']=np.where(world['value'] == 0, 0, np.exp(np.log(world['value']) + world['bayesian_update'] ))

In [ ]:
integral=world['numerator'].sum()
world['a_posteriori_distribution']=world['numerator']/integral
integral

98873995047.30652

In [53]:
world.to_csv('wolrd_pn.csv')